In [1]:
import pandas as pd
import numpy as np
import sys
import time

sys.path.append('../../../')
from utils import modelling as mod
from utils import utils
from utils import paths
import wandb

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score, cross_validate

In [2]:
df = pd.read_csv('/Users/tudor/Documents/phd/coding/project_CF/data/final_df_aberdeen.csv')
weights = mod.CalcClassWeights(df['floods'])
df_to_analyse = df.drop(columns=['Unnamed: 0.1', 'time', 'Unnamed: 0', 'floods', 'time_ok', 't'])

X = df_to_analyse.drop(columns=['floods_x4'])
y = df_to_analyse['floods_x4']

X_train = X.iloc[0:int(len(X) *.8)] 
X_test = X.iloc[int(len(X) *.8):int(len(X) *.9)]
X_val = X.iloc[int(len(X) *.9):]

y_train = y.iloc[0:int(len(y) *.8)]
y_test = y.iloc[int(len(y) *.8):int(len(y) *.9)]
y_val = y.iloc[int(len(y) *.9):]

In [3]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

In [19]:
dectree = DecisionTreeClassifier(
    criterion = 'log_loss',
    splitter='best',
    max_depth=200,
    min_samples_split = 50,
    min_samples_leaf= 50,
    random_state=42,
    class_weight=weights,
)

In [20]:
dectree.fit(X_train,y_train)

InvalidParameterError: The 'max_features' parameter of DecisionTreeClassifier must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.

In [17]:
y_pred = dectree.predict(X_test)

In [18]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.4999571183533448